In [ ]:
%load_ext autoreload
%autoreload
from pathlib import Path
import json
from src.evaluation.pycocotools.coco import COCO
from src.evaluation.pycocotools.cocoeval import COCOeval, Params

In [ ]:
ann_path = '/datagrid/personal/kuntluka/dental_rtg3/annotations.json'
# preds_path = 'predictions_faster_rcnn_res50.json'
# preds_path = 'predictions_faster_rcnn_res101.json'
preds_path = 'predictions_yolov5_medium.json'
# preds_path ='predictions_yolov5_large.json'
# preds_path = 'predictions_efficientdet_d4.json'
# preds_path2 = 'predictions_retinanet_swint.json'
# preds_path = 'predictions_retinanet_resnet50.json'

In [ ]:
with open('predictions_4000/'+preds_path) as f: 
    data = json.load(f)

In [ ]:
pred_path = Path('predictions_4000')

In [ ]:
preds = list(pred_path.iterdir())

In [ ]:
preds[0].name

In [ ]:
from src.utils.conver_to_coco import to_coco
from src.evaluation.prediction_evaluation import PredictionEval

with open(ann_path, 'r') as f:
    ann_file = json.load(f)

with open(pred_path/ preds_path, 'r') as f:
    preds_dict = json.load(f)

# with open(preds_path2, 'r') as f:
#     preds_dict2 = json.load(f)

# with open(preds_path3, 'r') as f:
#     preds_dict3 = json.load(f)

preds_coco, names = to_coco(str(pred_path /preds_path), ann_path)

pred_eval = PredictionEval()
pred_eval.load_data_coco_files(ann_path, preds_coco, names)

with open('metadata.json', 'r') as f:
    meta_data = json.load(f)

In [ ]:
with open('ens_all_preds.json') as f:
    data = json.load(f)

In [ ]:
preds_coco, names = to_coco(data, ann_path)

In [ ]:
pred_eval.load_predictions(preds_coco)

In [ ]:
p,r,c = pred_eval.precision_recall_score(stage='test')

In [ ]:
for prec, 

In [ ]:
np.max(2*p*r/(p+r))

In [ ]:
c[67]

In [ ]:
den = TP[67] + FP[67] + FN[67] + (1061 - FN[67] )

In [ ]:
acc = (TP[67] + (1061 - FN[67])) / den

In [ ]:
acc

In [ ]:
r

In [ ]:
c

In [ ]:
import numpy as np

In [ ]:
num_preds_arr=[]
for confidence in c:
    num_preds = 0
    for key, value in data.items(): 
        if value['stage'] != 'test': 
            continue 
        tmp_s = np.asarray(value['scores']) 
        num_preds += np.count_nonzero(tmp_s >= confidence) 
    num_preds_arr.append(num_preds)

In [ ]:
TP = []
FP = []
for prec, num in zip(p, num_preds_arr):
    TP.append(prec*num)
    FP.append((1-prec)*num)

In [ ]:
rc = []
FN = []
for rec in r:
    rc.append(rec*1061)
    FN.append((1-rec)*1061)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')
tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "arial",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 11,
    "font.size": 11,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 9,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9
}
plt.rcParams.update(tex_fonts)

def set_size(width_pt, fraction=1, subplots=(1,1)):
    fig_width_pt = width_pt * fraction
    inches_per_pt = 1 / 72.27
    golden_ratio = (5**.5 - 1) / 2
    fig_width_in = fig_width_pt * inches_per_pt
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])
    return (fig_width_in, fig_height_in)

In [ ]:
TP = np.asarray(TP)
FP = np.asarray(FP)
FN = np.asarray(FN)

In [ ]:
fig, ax = plt.subplots(figsize=set_size(407))
ax.plot(r[:90], FP[:90]/599)
ax.set_xlim([0,1])
ax.set_ylabel('FP detections per image')
ax.set_xlabel('Recall')
fig.savefig('fp_recall.pdf', format='pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=set_size(407))
ax.plot(p[:91], FN[:91]/1061*100)
ax.set_xlim([0.3,1])
ax.set_ylabel("Non-detected dental caries [\%]")
ax.set_xlabel('Precision')
fig.savefig('nondet_precision.pdf', format='pdf', bbox_inches='tight')

In [ ]:
num_preds_arrk

In [ ]:
pred_eval.cocoEval.evaluate()
pred_eval.cocoEval.accumulate()

In [ ]:
pred_eval.img

In [ ]:
meta = {}
meta['test_ids'] = names['test']
meta['train_ids'] = names['train']
meta['val_ids'] = names['val']

In [ ]:
meta['train_files'] = [pred_eval.img_id2name[id] for id in meta['train_ids']]
meta['test_files'] = [pred_eval.img_id2name[id] for id in meta['test_ids']]
meta['val_files'] = [pred_eval.img_id2name[id] for id in meta['val_ids']]

In [ ]:
with open('metadata4000.json', 'w') as f:
    json.dump(meta, f)

In [ ]:
names['type']

In [ ]:
names.keys()

In [ ]:
meta_data.keys()

In [ ]:
meta_data['train_ids'] == names['train']

In [ ]:
print(len(meta_data['test_ids']))
print(len(names['test']))

In [ ]:
len(meta_data['test_ids'] + meta_data['train_ids'] + meta_data['val_ids'])

In [ ]:
def evaluate_ckpts(preds_paths, annotations_path, metadata_path='metadata.json', out_name="results.txt"):

    with open(annotations_path, 'r') as f:
        ann_file = json.load(f)
    
    out_file_path = Path(out_name)
    out_file_path.unlink(missing_ok=True)
    
    for pred_path in preds_paths:
        with open(pred_path) as f:
            preds_dict = json.load(f)
        preds_coco, names = to_coco(preds_dict, ann_file)
        pred_eval = PredictionEval()
        pred_eval.load_data_coco_files(annotations_path, preds_coco, names)
        name = str(pred_path).replace('.json','').replace('predictions_', '')
        text, prec = pred_eval.get_data(name)

        with open(out_name, 'a') as out_file:
            out_file.write(f"{text}\n{prec}\n\n")


In [ ]:
# evaluate_ckpts([ 'predictions_yolov5_medium.json', 'predictions_yolov5_large.json'], ann_path)
preds = list(Path('predictions_4000').iterdir())
evaluate_ckpts(preds, ann_path)

In [ ]:
meta_data.keys()

In [ ]:
import numpy as np
from src.evaluation.boxes_fusion.ensemble_boxes_wbf import weighted_boxes_fusion
boxes1 = []
boxes2 = []
scores1 =[]
scores2 =[]
dict_out = {}

for img_name in meta_data['test_files']:

    boxes1 = []
    boxes2 = []
    boxes3 = []
    d1 = preds_dict[img_name]
    for bbox in d1['bboxes']:
        x1,y1,x2,y2 = bbox
        boxes1.append([x1/1068, y1/847, x2/1068, y2/847])
    d2 = preds_dict2[img_name]

    for bbox in d2['bboxes']:
        x1,y1,x2,y2 = bbox
        boxes2.append([x1/1068, y1/847, x2/1068, y2/847])

    d3 = preds_dict3[img_name]

    for bbox in d3['bboxes']:
        x1,y1,x2,y2 = bbox
        boxes3.append([x1/1068, y1/847, x2/1068, y2/847])
    
    boxes = [boxes1, boxes2, boxes3]
    scores = [d1['scores'], d2['scores'], d3['scores']]
    labels = [d1['labels'], d2['labels'], d3['labels']]
    # labels = np.zeros_like(np.asarray(scores)).tolist()
    # labels = [[0 for bboxes in boxes] for box in boxes]
    weights = [3, 2, 2]
    # weights = [[1,1,1], [2,2,2], [3,3,3]]
    iou_thr = 0.55
    skip_box_thr = 0.001
    sigma = 0.1

    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

    boxes_out = [] 
    for bbox in boxes:
        x1, y1, x2, y2 = bbox
        boxes_out.append([x1*1068, y1 * 847, x2 * 1068, y2 * 847])
    dict_out[img_name] = {'bboxes' : boxes_out, 'scores' : scores, 'labels' : labels, 'stage':'test'}
    # for box, score in zip(d1['bboxes'], d1['scores']):

    # boxes1=

In [ ]:
preds_coco, _ = to_coco(dict_out, ann_path)
pred_eval.load_data_coco_files(ann_path, preds_coco['annotations'], names)

In [ ]:
text, prec = pred_eval.get_data(preds_path)

In [ ]:
text

In [ ]:
prec